Model that we are using:

*   ResNet
*   AlexNet (Kevin)
*   DenseNet
*   VGG







In [ ]:
!rm 17flowers_labeled.zip
!rm -R train_data
!rm -R 17flowers
!wget https://filebin.net/trsh6uhonv07jvum/17flowers_labeled.zip
!unzip 17flowers_labeled

rm: cannot remove '17flowers_labeled.zip': No such file or directory
rm: cannot remove 'train_data': No such file or directory
rm: cannot remove '17flowers': No such file or directory
--2022-10-30 08:56:56--  https://filebin.net/trsh6uhonv07jvum/17flowers_labeled.zip
Resolving filebin.net (filebin.net)... 185.47.40.36, 2a02:c0:2f0:700:f816:3eff:feac:c605
Connecting to filebin.net (filebin.net)|185.47.40.36|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://situla.bitbit.net/filebin/a8d26edf4e42db9b3268facc077e9060f1a409d9d070b4910821aa6494313963/8faef63e94ac669c6f25f634e90a779344901ed59b0f4e9a4c0e6d22bab8a6f4?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Credential=HZXB1J7T0UN34UN512IW%2F20221030%2Fus-east-1%2Fs3%2Faws4_request&X-Amz-Date=20221030T085657Z&X-Amz-Expires=30&X-Amz-SignedHeaders=host&response-cache-control=max-age%3D30&response-content-disposition=filename%3D%2217flowers_labeled.zip%22&response-content-type=application%2Fzip&X-Amz-Signature=4a5

In [ ]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
import os
def walk_through_dir(dir_path):
  """
  Walks through dir_path returning its contents.
  Args:
    dir_path (str or pathlib.Path): target directory
  
  Returns:
    A print out of:
      number of subdiretories in dir_path
      number of images (files) in each subdirectory
      name of each subdirectory
  """
  for dirpath, dirnames, filenames in os.walk(dir_path):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")


walk_through_dir("17flowers")

There are 17 directories and 2 images in '17flowers'.
There are 0 directories and 80 images in '17flowers/Tigerlily'.
There are 0 directories and 80 images in '17flowers/Iris'.
There are 0 directories and 80 images in '17flowers/Daisy'.
There are 0 directories and 80 images in '17flowers/LilyValley'.
There are 0 directories and 80 images in '17flowers/Dandelion'.
There are 0 directories and 80 images in '17flowers/Sunflower'.
There are 0 directories and 80 images in '17flowers/Colts'Foot'.
There are 0 directories and 80 images in '17flowers/Cowslip'.
There are 0 directories and 80 images in '17flowers/Daffodil'.
There are 0 directories and 80 images in '17flowers/Snowdrop'.
There are 0 directories and 80 images in '17flowers/Crocus'.
There are 0 directories and 80 images in '17flowers/Windflower'.
There are 0 directories and 80 images in '17flowers/Buttercup'.
There are 0 directories and 80 images in '17flowers/Fritillary'.
There are 0 directories and 80 images in '17flowers/Pansy'.
Th

In [ ]:
!pip install split-folders

import splitfolders

splitfolders.ratio("17flowers", # The location of dataset
                   output="train_data", # The output location
                   seed=42, # The number of seed
                   ratio=(.8, .2), # The ratio of splited dataset
                   group_prefix=None, # If your dataset contains more than one file like ".jpg", ".pdf", etc
                   move=False # If you choose to move, turn this into True
                   )

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Copying files: 1360 files [00:00, 6071.88 files/s]


In [ ]:
image_path = "train_data"
# Setup train and testing paths
train_dir = image_path + "/train"
test_dir = image_path + "/val"

train_dir, test_dir

('train_data/train', 'train_data/val')

In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import time
import os

In [ ]:
print(torch.cuda.is_available())

True


In [ ]:
# Write transform for image
data_transform = transforms.Compose([
    # Resize the images to 64x64
    transforms.Resize(size=(224, 224)),
    # Flip the images randomly on the horizontal
    transforms.RandomHorizontalFlip(p=0.5), # p = probability of flip, 0.5 = 50% chance
    # Turn the image into a torch.Tensor
    transforms.ToTensor() # this also converts all pixel values from 0 to 255 to be between 0.0 and 1.0 
])

# Use ImageFolder to create dataset(s)
from torchvision import datasets
train_data = datasets.ImageFolder(root=train_dir, # target folder of images
                                  transform=data_transform, # transforms to perform on data (images)
                                  target_transform=None) # transforms to perform on labels (if necessary)

test_data = datasets.ImageFolder(root=test_dir, 
                                 transform=data_transform)

print(f"Train data:\n{train_data}\nTest data:\n{test_data}")

Train data:
Dataset ImageFolder
    Number of datapoints: 1088
    Root location: train_data/train
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=None)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
           )
Test data:
Dataset ImageFolder
    Number of datapoints: 272
    Root location: train_data/val
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=None)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
           )


In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride = 1, downsample = None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Sequential(
                        nn.Conv2d(in_channels, out_channels, kernel_size = 3, stride = stride, padding = 1),
                        nn.BatchNorm2d(out_channels),
                        nn.ReLU())
        self.conv2 = nn.Sequential(
                        nn.Conv2d(out_channels, out_channels, kernel_size = 3, stride = 1, padding = 1),
                        nn.BatchNorm2d(out_channels))
        self.downsample = downsample
        self.relu = nn.ReLU()
        self.out_channels = out_channels
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.conv2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

In [ ]:
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes = 17):
        super(ResNet, self).__init__()
        self.inplanes = 64
        self.conv1 = nn.Sequential(
                        nn.Conv2d(3, 64, kernel_size = 7, stride = 2, padding = 3),
                        nn.BatchNorm2d(64),
                        nn.ReLU())
        self.maxpool = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
        self.layer0 = self._make_layer(block, 64, layers[0], stride = 1)
        self.layer1 = self._make_layer(block, 128, layers[1], stride = 2)
        self.layer2 = self._make_layer(block, 256, layers[2], stride = 2)
        self.layer3 = self._make_layer(block, 512, layers[3], stride = 2)
        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(512, num_classes)
        
    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes:
            
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes, kernel_size=1, stride=stride),
                nn.BatchNorm2d(planes),
            )
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)
    
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool(x)
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

**Load** Data

In [ ]:
# Turn train and test Datasets into DataLoaders
from torch.utils.data import DataLoader
train_dataloader = DataLoader(dataset=train_data, 
                               batch_size=10, # how many samples per batch?
                              num_workers=1, # how many subprocesses to use for data loading? (higher = more)
                              shuffle=True) # shuffle the data?

test_dataloader = DataLoader(dataset=test_data, 
                             batch_size=10, 
                             num_workers=1, 
                             shuffle=False) # don't usually need to shuffle testing data

train_dataloader, test_dataloader

(<torch.utils.data.dataloader.DataLoader at 0x7f2b5e664950>,
 <torch.utils.data.dataloader.DataLoader at 0x7f2b5e664990>)

In [ ]:
classes = train_data.classes
num_classes = len(classes)
num_epochs = 50
batch_size = 10
learning_rate = 0.01

model = ResNet(ResidualBlock, [3, 4, 6, 3]).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.0001, momentum = 0.1)  
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5) # Decay strategy of the learning rate

# Train the model
total_step = len(train_dataloader)

Start Trainning

In [ ]:
import gc
total_step = len(train_dataloader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_dataloader):  
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        del images, labels, outputs
        torch.cuda.empty_cache()
        gc.collect()

    print ('Epoch [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, loss.item()))
            
    # Validation
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_dataloader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, outputs
    
        print('Accuracy of the network on the {} validation images: {} %'.format(200, 100 * correct / total)) 

Epoch [1/2000], Loss: 3.0139
Accuracy of the network on the 200 validation images: 4.411764705882353 %
Epoch [2/2000], Loss: 2.9377
Accuracy of the network on the 200 validation images: 5.514705882352941 %
Epoch [3/2000], Loss: 2.6533
Accuracy of the network on the 200 validation images: 6.25 %
Epoch [4/2000], Loss: 2.4285
Accuracy of the network on the 200 validation images: 5.882352941176471 %


KeyboardInterrupt: ignored

In [ ]:
classes = train_data.classes
train_size = 1088
def train_test(model, criterion, optimizer, scheduler, num_epochs):
    train_loss = []
    train_accuracy = []
    val_loss = []
    val_accuracy = []
    history = dict()
    model.train()
    for epoch in range(num_epochs):
        running_training_loss = 0.0
        running_training_accuracy = 0.0
        iteration_training_loss = 0.0
        total_training_predictions = 0

        start_time = time.time()
        for i, data in enumerate(train_dataloader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_training_loss += loss.item()*inputs.size(0)

            _, predicted = torch.max(outputs.data, 1)
            total_training_predictions += labels.size(0)
            running_training_accuracy += (predicted == labels).sum().item()
            iteration_training_loss += loss.item()
            # if (i+1) % 100 == 0:
            #     print('Epoch:[%d]-Iteration:[%d], training loss: %.3f' %
            #           (epoch + 1,i+1,iteration_training_loss/(i+1)))
            
        # Evaluate model after epoch
        #TODO - add evaluation step to produce graph 


        end_time = time.time()
        print('Time cost of one epoch: [%d]s' % (end_time-start_time))

        epoch_training_accuracy = running_training_accuracy / train_size*100
        # print("eooch training accuracy", epoch_training_accuracy)
        # print("running training accuracy", running_training_accuracy)
        # print("train size", train_size)
        epoch_training_loss = running_training_loss / train_size

        print('Epoch:[%d], training accuracy: %.1f, training loss: %.3f' %
              (epoch + 1,epoch_training_accuracy, epoch_training_loss))

        train_loss.append(epoch_training_loss)
        train_accuracy.append(epoch_training_accuracy)

        scheduler.step()

    print('Finished Training')

    history['train_loss'] = train_loss
    history['train_accuracy'] = train_accuracy

    correct = 0
    total = 0
    model.eval()

    # prepare to count predictions for each class
    correct_pred = {classname: 0 for classname in classes}
    total_pred = {classname: 0 for classname in classes}
    test_accuracy = {classname: 0 for classname in classes}

    # again no gradients needed
    with torch.no_grad():
        for data in test_dataloader:
            images, labels = data
            images, labels = images.cuda(), labels.cuda()
            outputs = model_ft(images)
            _, predictions = torch.max(outputs, 1)
            # collect the correct predictions for each class
            for label, prediction in zip(labels, predictions):
                if label == prediction:
                    correct_pred[classes[label]] += 1
                total_pred[classes[label]] += 1

    # print accuracy for each class
    for classname, correct_count in correct_pred.items():
        accuracy = 100 * float(correct_count) / total_pred[classname]
        print("Accuracy for class {:5s} is: {:.1f} %".format(classname, accuracy))
        test_accuracy[classname] = accuracy / 100

    return history, test_accuracy

In [ ]:
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

if __name__ == '__main__':
    end = time.time()
    model_ft = ResNet(ResidualBlock, [3, 4, 6, 3]).to(device) # Model initialization
    # print(model_ft.network)
    criterion = nn.CrossEntropyLoss() # Loss function initialization

    # TODO: Adjust the following hyper-parameters: initial learning rate, decay strategy of the learning rate, number of training epochs
    optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.01) # The initial learning rate is 1e-3

    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.5) # Decay strategy of the learning rate
    
    
    history, accuracy = train_test(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=50) # The number of training epochs is 25
    
    print("time required %.2fs" %(time.time() - end))
    print("Average verification accuracy %.2fpercent"%(sum(accuracy.values())/17*100))

Time cost of one epoch: [11]s
Epoch:[1], training accuracy: 8.9, training loss: 3.325
Time cost of one epoch: [11]s
Epoch:[2], training accuracy: 11.4, training loss: 2.621
Time cost of one epoch: [11]s
Epoch:[3], training accuracy: 13.6, training loss: 2.451
Time cost of one epoch: [11]s
Epoch:[4], training accuracy: 19.0, training loss: 2.317
Time cost of one epoch: [11]s
Epoch:[5], training accuracy: 17.8, training loss: 2.292
Time cost of one epoch: [11]s
Epoch:[6], training accuracy: 21.1, training loss: 2.227
Time cost of one epoch: [11]s
Epoch:[7], training accuracy: 23.3, training loss: 2.161
Time cost of one epoch: [11]s
Epoch:[8], training accuracy: 29.1, training loss: 2.040
Time cost of one epoch: [11]s
Epoch:[9], training accuracy: 30.6, training loss: 1.953
Time cost of one epoch: [11]s
Epoch:[10], training accuracy: 35.4, training loss: 1.836
Time cost of one epoch: [11]s
Epoch:[11], training accuracy: 41.0, training loss: 1.655
Time cost of one epoch: [11]s
Epoch:[12], 